In [176]:
# Growth Area Coordinates
import pandas as pd
import requests
import json
import hvplot.pandas
from config import geoapify_key
print (geoapify_key)

d5037d604961461b9a3e13edebf3d8ec


In [177]:
growth_data = pd.read_csv("Output/grmelbourne_growth_byarea.csv")
ptv_data = pd.read_csv("Output/PTV_suburb.csv")
suburb_list = pd.DataFrame (growth_data[["Area Name"]])
# suburb_list=suburb_list.sample(5)


In [178]:
# !!!!!!!!!!!!!!!
# In stead of running this script you can import the CSV for the rest




base_url = "https://api.geoapify.com/v1/geocode/search"
params = {
    "apiKey":geoapify_key,
    "format":"json"
            }

for index, row in suburb_list.iterrows():
    area = row["Area Name"] + ",Vic,Australia"
    params["text"] = f"{area}"
    response = requests.get(base_url, params=params).json()
    # response = response.json()
    try:
        suburb_list.loc[index, "Lat"] = response["results"][0]["lat"]
        suburb_list.loc[index, "Lon"] = response["results"][0]["lon"]
        suburb_list.loc[index, "place_id"] = response["results"][0]["place_id"]
        suburb_list.loc[index, "addr_1"] = response["results"][0]["address_line1"]
        suburb_list.loc[index, "addr-2"] = response["results"][0]["address_line1"]
        suburb_list.loc[index, "Confidence"] = response["results"][0]["rank"]["confidence"]
        suburb_list.loc[index, "Municipality"] = response["results"][0]["municipality"]
        suburb_list.loc[index, "Suburb_geoapify"] = response["results"][0]["suburb"]
        suburb_list.loc[index, "bbox_lat1"] = response["results"][0]["bbox"]["lat1"]
        suburb_list.loc[index, "bbox_lat2"] = response["results"][0]["bbox"]["lat2"]
        suburb_list.loc[index, "bbox_lon1"] = response["results"][0]["bbox"]["lon1"]
        suburb_list.loc[index, "bbox_lon2"] = response["results"][0]["bbox"]["lon2"]

    except KeyError as e:
       
        print(f"{e.args[0]} results not found for {suburb_list.loc[index, 'Area Name']}")
    
# print(json.dumps(response, indent=4, sort_keys=True))



suburb results not found for Bunyip
suburb results not found for Garfield
suburb results not found for Cockatoo
suburb results not found for Flinders
suburb results not found for Gisborne
suburb results not found for Hastings
municipality results not found for Hillside
suburb results not found for Koo Wee Rup
suburb results not found for Macedon
municipality results not found for Point Nepean
suburb results not found for Riddells Creek
suburb results not found for Romsey
suburb results not found for Sunbury
suburb results not found for Whittlesea


In [182]:
# drop if confidence level is less than 1
suburb_list=suburb_list[suburb_list["Confidence"]==1]

# drop outliers
remove =["Hillside", "Bunyip", "Garfield","Point Nepean"]
suburb_list = suburb_list[~suburb_list["Area Name"].isin(remove)]
# rename greenvale - bulla to greenvale (will miss 84 bulla ptv trips)
suburb_list['Area Name'] = suburb_list['Area Name'].replace("Greenvale - Bulla","Greenvale")

growth_mapped_geopafy =pd.merge(growth_data, suburb_list, how ="right", on="Area Name")
# growth_mapped_geopafy['Total 2016']=growth_mapped_geopafy['Total 2016'].str.replace(',','')
# growth_mapped_geopafy['Total 2016'] = growth_mapped_geopafy['Total 2016'].astype('float')
# growth_mapped_geopafy['Total 2021']=growth_mapped_geopafy['Total 2021'].str.replace(',','')
# growth_mapped_geopafy['Total 2021'] = growth_mapped_geopafy['Total 2021'].astype('float')
# growth_mapped_geopafy['Total 2026']=growth_mapped_geopafy['Total 2026'].str.replace(',','')
# growth_mapped_geopafy['Total 2026'] = growth_mapped_geopafy['Total 2026'].astype('float')
# growth_mapped_geopafy['Total 2031']=growth_mapped_geopafy['Total 2031'].str.replace(',','')
# growth_mapped_geopafy['Total 2031'] = growth_mapped_geopafy['Total 2031'].astype('float')
# growth_mapped_geopafy['Total 2036']=growth_mapped_geopafy['Total 2036'].str.replace(',','')
# growth_mapped_geopafy['Total 2036'] = growth_mapped_geopafy['Total 2036'].astype('float')
# growth_mapped_geopafy['Total Growth']=growth_mapped_geopafy['Total Growth'].str.replace(',','')
# growth_mapped_geopafy['Total Growth'] = growth_mapped_geopafy['Total Growth'].astype('float')


# rename greenvale - bulla to greenvale (will miss 84 bulla ptv trips)



In [183]:
# save csv file
growth_mapped_geopafy.to_csv("Output/growth_mapped_geopafy.csv")

In [184]:
# import csv rather than running api above
# growth_mapped_geopafy = pd.read_csv("Output/growth_mapped_geopafy.csv")
# growth_mapped_geopafy.dtypes
# growth_mapped_geopafy['Total 2016']=growth_mapped_geopafy['Total 2016'].str.replace(',','')
# growth_mapped_geopafy['Total 2016'] = growth_mapped_geopafy['Total 2016'].astype('float')
# growth_mapped_geopafy['Total 2021']=growth_mapped_geopafy['Total 2021'].str.replace(',','')
# growth_mapped_geopafy['Total 2021'] = growth_mapped_geopafy['Total 2021'].astype('float')
# growth_mapped_geopafy['Total 2026']=growth_mapped_geopafy['Total 2026'].str.replace(',','')
# growth_mapped_geopafy['Total 2026'] = growth_mapped_geopafy['Total 2026'].astype('float')
# growth_mapped_geopafy['Total 2031']=growth_mapped_geopafy['Total 2031'].str.replace(',','')
# growth_mapped_geopafy['Total 2031'] = growth_mapped_geopafy['Total 2031'].astype('float')
# growth_mapped_geopafy['Total 2036']=growth_mapped_geopafy['Total 2036'].str.replace(',','')
# growth_mapped_geopafy['Total 2036'] = growth_mapped_geopafy['Total 2036'].astype('float')
# growth_mapped_geopafy['Total Growth']=growth_mapped_geopafy['Total Growth'].str.replace(',','')
# growth_mapped_geopafy['Total Growth'] = growth_mapped_geopafy['Total Growth'].astype('float')

# growth_mapped_geopafy['Total 2016']=growth_mapped_geopafy['Total 2016'].replace(',','')
# growth_mapped_geopafy['Total 2016'] = growth_mapped_geopafy['Total 2016'].astype('float')
# growth_mapped_geopafy['Total 2021']=growth_mapped_geopafy['Total 2021'].replace(',','')
# growth_mapped_geopafy['Total 2021'] = growth_mapped_geopafy['Total 2021'].astype('float')
# growth_mapped_geopafy['Total 2026']=growth_mapped_geopafy['Total 2026'].replace(',','')
# growth_mapped_geopafy['Total 2026'] = growth_mapped_geopafy['Total 2026'].astype('float')
# growth_mapped_geopafy['Total 2031']=growth_mapped_geopafy['Total 2031'].replace(',','')
# growth_mapped_geopafy['Total 2031'] = growth_mapped_geopafy['Total 2031'].astype('float')
# growth_mapped_geopafy['Total 2036']=growth_mapped_geopafy['Total 2036'].replace(',','')
# growth_mapped_geopafy['Total 2036'] = growth_mapped_geopafy['Total 2036'].astype('float')
# growth_mapped_geopafy['Total Growth']=growth_mapped_geopafy['Total Growth'].replace(',','')
# growth_mapped_geopafy['Total Growth'] = growth_mapped_geopafy['Total Growth'].astype('float')

# growth_mapped_geopafy = growth_mapped_geopafy[['Total 2016', 'Total 2021','Total 2026','Total 2031','Total 2036']].astype(float)
# # growth_mapped_geopafy[['Total 2016', 'Total 2021','Total 2026','Total 2031','Total 2046']] = growth_mapped_geopafy[['Total 2016', 'Total 2021','Total 2026','Total 2031','Total 2046']].apply(pd.to_numeric)
# growth_mapped_geopafy = growth_mapped_geopafy.convert_dtypes()
growth_mapped_geopafy.dtypes

Unnamed: 0         float64
Area Name           object
Total 2016         float64
Total 2021         float64
Total 2026         float64
Total 2031         float64
Total 2036         float64
GCCSA               object
Total Growth       float64
Total Growth %     float64
Lat                float64
Lon                float64
place_id            object
addr_1              object
addr-2              object
Confidence         float64
Municipality        object
Suburb_geoapify     object
bbox_lat1          float64
bbox_lat2          float64
bbox_lon1          float64
bbox_lon2          float64
dtype: object

In [204]:
map_plot1 = growth_mapped_geopafy.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 400,
    size = "Total Growth",
    scale = 0.04,
    color = "Area Name"
)

map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (Area Name,Total Growth)

In [203]:
total_growth_fromscsv = growth_data["Total Growth"].sum()
# print(total_growth_fromscsv)
# growth_data['Total Growth']=growth_data['Total Growth'].str.replace(',','')
# growth_data['Total 2016'] = growth_data['Total 2016'].astype('float')
total_growth_mapped = growth_mapped_geopafy["Total Growth"].sum()
# print (total_growth_mapped)
percentage_growth_mapped = (total_growth_mapped/total_growth_fromscsv)*100
percentage_growth_mapped = f"{percentage_growth_mapped:.2f}%"
total_growth_fromscsv=f"{total_growth_fromscsv:,.0f}"
# total_growth_froms_csv = f"{total_growth_froms_csv:,}"

print (f"We were able to map {percentage_growth_mapped} of the total growth of {total_growth_fromscsv} reported by Victoria")

We were able to map 97.36% of the total growth of 2,120,566 reported by Victoria


In [213]:

suburb_growth_bins=growth_mapped_geopafy.copy()
growth_bins=[0,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 10]
labels=["<30%","$40%","50%", "60%", "70%", "80%","90%",">100%"]
suburb_growth_bins["Growth % per suburb"]=pd.cut(suburb_growth_bins['Total Growth %'], growth_bins, labels=labels)

suburb_growth_bins_count = suburb_growth_bins.groupby(["Growth % per suburb"]).count()["Total Growth %"]
suburb_growth_bins_total = suburb_growth_bins.groupby(["Growth % per suburb"]).sum()["Total Growth"]
suburb_growth_bins_average = suburb_growth_bins.groupby(["Growth % per suburb"]) ["Total Growth"].mean()
suburb_growth_bins_perc = suburb_growth_bins.groupby(["Growth % per suburb"]) ["Total Growth %"].mean()

suburb_growth_bins = pd.DataFrame({"Number of growth areas":suburb_growth_bins_count,"Total Growth":suburb_growth_bins_total ,"Average Growth":suburb_growth_bins_average, "Average Growth %":suburb_growth_bins_perc })
suburb_growth_bins

,Number of growth areas,Total Growth,Average Growth,Average Growth %
Growth % per suburb,,,,
<30%,167,451320.0,2702.514970,0.161377
$40%,34,210914.0,6203.352941,0.347353
50%,20,154937.0,7746.850000,0.446500
60%,17,234057.0,13768.058824,0.545882
70%,8,74420.0,9302.500000,0.648750
80%,6,98348.0,16391.333333,0.763333
90%,2,27480.0,13740.000000,0.835000
>100%,18,696330.0,38685.000000,2.116667


In [215]:
over_100perc_growth = growth_mapped_geopafy.loc[growth_mapped_geopafy["Total Growth %"]>=1]
map_plot2 = over_100perc_growth.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Total Growth",
    scale = 0.04,
    color = "Area Name"
)
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (Area Name,Total Growth)

In [216]:
ptv_trip = pd.read_csv("Resources/P1_trip_frequency_adjRAVV.csv")
ptv_trip.head()

top_growth_ptv_merge = pd.merge(over_100perc_growth, ptv_trip, how = "left", left_on="Area Name", right_on="Suburb")
top_growth_ptv_merge

,Unnamed: 0,Area Name,Total 2016,Total 2021,Total 2026,Total 2031,Total 2036,GCCSA,Total Growth,Total Growth %,...,bbox_lat2,bbox_lon1,bbox_lon2,Suburb,bus,train,tram,Grand Total,trips_per_stop,Unnamed: 6
0,NaN,Werribee,44702.0,57757.0,72036.0,87180.0,102930.0,2GMEL,58228.0,1.30,...,-37.870364,144.578968,144.718944,Werribee,7922.0,146.0,0.0,8068,35.541850,NaN
1,1018.0,Beaconsfield - Officer,13612.0,25467.0,36250.0,43984.0,49252.0,2GMEL,35640.0,2.62,...,-38.013286,145.375867,145.441971,Beaconsfield - Officer,776.0,596.0,NaN,1372,31.906977,43.0
2,1034.0,Cranbourne East,26754.0,60536.0,85547.0,105606.0,118601.0,2GMEL,91847.0,3.43,...,-38.084514,145.288241,145.324895,Cranbourne East,3990.0,NaN,NaN,3990,83.125000,NaN
3,1036.0,Cranbourne South,9116.0,16849.0,28663.0,41613.0,54603.0,2GMEL,45487.0,4.99,...,-38.121693,145.221735,145.291248,Cranbourne South,624.0,NaN,NaN,624,89.142857,NaN
4,115.0,Docklands,11832.0,18970.0,24820.0,27256.0,29157.0,2GMEL,17325.0,1.46,...,-37.809249,144.920056,144.955992,Docklands,456.0,NaN,6400.0,6856,285.666667,NaN
5,1083.0,Footscray,17343.0,23424.0,32211.0,39266.0,44492.0,2GMEL,27149.0,1.57,...,-37.786363,144.879280,144.916504,Footscray,9398.0,988.0,1136.0,11522,100.191304,NaN
6,969.0,Greenvale,17123.0,21905.0,27114.0,31463.0,35127.0,2GMEL,18004.0,1.05,...,-37.603468,144.833910,144.915412,Greenvale,2371.0,NaN,NaN,2371,35.924242,NaN
7,NaN,Melton South,43803.0,55584.0,68449.0,81710.0,95148.0,2GMEL,51345.0,1.17,...,-37.690213,144.549541,144.584934,Melton South,1952.0,NaN,NaN,1952,41.531915,NaN
8,978.0,Mickleham,3580.0,18538.0,33525.0,47112.0,58334.0,2GMEL,54754.0,15.29,...,-37.502067,144.848330,144.950012,Mickleham,2764.0,NaN,NaN,2764,58.808511,NaN
9,120.0,North Melbourne,22106.0,29306.0,33979.0,40364.0,45392.0,2GMEL,23286.0,1.05,...,-37.787463,144.935279,144.958510,North Melbourne,1383.0,460.0,5048.0,6891,127.611111,NaN
